In [26]:
import mysql.connector
from mysql.connector import Error as MySQLError
from mysql.connector import connection as MySQLConnection
import os
import time
from typing import List, Dict

MYSQL_HOST = "127.0.0.1"
MYSQL_PORT = 3308
MYSQL_USER = os.environ["MYSQL_USER"]
MYSQL_PASSWORD = os.environ["MYSQL_PASSWORD"]
MYSQL_DATABASE = os.environ["MYSQL_DATABASE"]

cur = None
conn = None
def query(q: str, n: int = None):
    global cur
    global conn
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()
    conn = mysql.connector.connect(host=MYSQL_HOST,
                                database=MYSQL_DATABASE,
                                user=MYSQL_USER,
                                password=MYSQL_PASSWORD,
                                port=MYSQL_PORT,)
    if not conn.is_connected():
        raise Exception("Could not connect")
    cur = conn.cursor()
    cur.execute(q)
    i = 0
    if n is not None:
        while i < n:
            try:
                row = next(cur)
                yield row
                i += 1
            except:
                break
    else:
        for row in cur:
            yield row

def create_insert_query(d: Dict, table: str):
    cols = list(d.keys())
    template = ",".join(["%s" for _ in cols])
    cols_str = ",".join(cols)
    ordered_vals = tuple([d[x] for x in cols])
    return f"INSERT INTO {table} ({cols_str}) VALUES ({template})", ordered_vals

def insert(d: List[Dict], table: str):
    global cur
    cur = conn.cursor()
    for i in d:
        q, v = create_insert_query(i, table)
        cur.execute(q, v)
    conn.commit()
    cur.close()

def get_cols(table: str):
    return (tuple([x[0] for x in (query(f"DESCRIBE {table}"))]))

In [27]:
print(get_cols("telegram.messages"))
template="special message content"
for row in query(f"""SELECT * FROM messages WHERE text LIKE "%{template}%"  """, 10):
    print(row)

In [39]:
list(query("SELECT COUNT(*) FROM messages"))

[(29259,)]